## Problem-2:  Programming Value Iteration

### (a)

In [1]:
import gymnasium as gym
import numpy as np
import imageio

In [2]:
# Create the FrozenLake environment
env = gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=False, render_mode='rgb_array')

# Define the maximum number of epochs
max_epochs = 500

# Define a function to print the environment layout
def print_env(env):
    if hasattr(env, 'desc'):
        for row in env.unwrapped.desc:
            print(row)

# Print the environment layout
print_env(env)

[b'S' b'F' b'F' b'F']
[b'F' b'H' b'F' b'H']
[b'F' b'F' b'F' b'H']
[b'H' b'F' b'F' b'G']


/home/pradeep/.local/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.desc to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.desc` for environment variables or `env.get_wrapper_attr('desc')` that will search the reminding wrappers.
  logger.warn(


In [3]:
def value_iteration(env, gamma=0.95, theta=1e-6):
    # Get the number of states and actions from the environment
    num_states = env.observation_space.n
    num_actions = env.action_space.n

    # Initialize the value function V to zeros for each state
    V = np.zeros(num_states)

    # Loop for a maximum of 1000 epochs (iterations)
    for epoch in range(1000):
        # Initialize the maximum change in value function to zero
        delta = 0

        # Iterate over all states in the environment
        for s in range(num_states):
            v = V[s]  # Store the current value for state s
            q_values = np.zeros(num_actions)  # Initialize an array to store Q-values for each action

            # Iterate over all possible actions
            for a in range(num_actions):
                # Iterate over possible outcomes of taking action a in state s
                for prob, next_state, reward, _ in env.unwrapped.P[s][a]:
                    # Update the Q-value for action a in state s using the Bellman equation
                    q_values[a] += prob * (reward + gamma * V[next_state])

            # Update the value function for state s with the maximum Q-value
            V[s] = max(q_values)

            # Update the maximum change in value function
            delta = max(delta, abs(v - V[s]))

        # If the maximum change in value function is below the specified threshold (theta), break the loop
        if delta < theta:
            print(f'Breaking at {epoch} epoch')
            break

    # Return the computed value function V
    return V

In [4]:
V_value_iter = value_iteration(env)

print('Value Function:')
print(V_value_iter.reshape(4,4))

Breaking at 6 epoch
Value Function:
[[0.77378094 0.81450625 0.857375   0.81450625]
 [0.81450625 0.         0.9025     0.        ]
 [0.857375   0.9025     0.95       0.        ]
 [0.         0.95       1.         0.        ]]


In [5]:
def policy_improvement(env, V, gamma=0.95):
    # Get the number of states and actions from the environment
    num_states = env.observation_space.n
    num_actions = env.action_space.n

    # Initialize the policy as an array of zeros
    policy = np.zeros(num_states)

    for s in range(num_states):
        q_values = np.zeros(num_actions)

        for a in range(num_actions):
            # Iterate over possible outcomes of taking action a in state s
            for prob, next_state, reward, _ in env.unwrapped.P[s][a]:
                # Calculate the expected future reward using the value function V
                q_values[a] += prob * (reward + gamma * V[next_state])

        # Choose the action that maximizes the expected future reward
        best_action = np.argmax(q_values)
        policy[s] = best_action

    return policy

In [6]:
def policy_evaluation(env, policy, gamma=0.95, theta=1e-6):
    # Get the number of states from the environment
    num_states = env.observation_space.n

    # Initialize the value function V to zeros for each state
    V = np.zeros(num_states)
    
    # Loop for a maximum of 1000 epochs (iterations)
    for epoch in range(1000):
        # Initialize the maximum change in value function to zero
        delta = 0

        # Iterate over all states in the environment
        for s in range(num_states):
            v = V[s]  # Store the current value for state s
            new_v = 0  # Initialize a variable to store the updated value for state s

            # Get the action to be taken in state s according to the policy
            a = policy[s]

            # Iterate over possible outcomes of taking action a in state s
            for prob, next_state, reward, _ in env.unwrapped.P[s][a]:
                # Update the new value for state s using the Bellman equation and the policy
                new_v += prob * (reward + gamma * V[next_state])

            # Update the value function for state s with the new value
            V[s] = new_v

            # Update the maximum change in value function
            delta = max(delta, abs(v - new_v))
        
        # If the maximum change in value function is below the specified threshold (theta), break the loop
        if delta < theta:
            break
    
    # Return the computed value function V
    return V

In [7]:

def policy_iteration(env, max_epochs=1000):
    # Get the number of states and actions from the environment
    num_states = env.observation_space.n
    num_actions = env.action_space.n

    # Initialize the value function V and the policy as arrays of zeros
    V = np.zeros(num_states)
    policy = np.zeros(num_states)

    for epoch in range(max_epochs):
        # Policy Evaluation: Compute the value function V for the current policy
        V = policy_evaluation(env, policy)

        # Policy Improvement: Find an improved policy based on the current value function
        new_policy = policy_improvement(env, V)

        # If the policy has not changed, break the loop (the optimal policy has been found)
        if (policy == new_policy).all():
            print(f'Breaking at epoch: {epoch}')
            break

        # Update the current policy with the new policy
        policy = new_policy

    # Return the computed value function V and the optimal policy
    return V, policy

In [8]:
V_policy_iter, policy_policy_iter = policy_iteration(env)

print('Environment Grid:')
print_env(env)

print('\nValue Function:\n', V_policy_iter.reshape(4,4))
print('\nPolicy:\n', policy_policy_iter.reshape(4,4))

Breaking at epoch: 6
Environment Grid:
[b'S' b'F' b'F' b'F']
[b'F' b'H' b'F' b'H']
[b'F' b'F' b'F' b'H']
[b'H' b'F' b'F' b'G']

Value Function:
 [[0.77378094 0.81450625 0.857375   0.81450625]
 [0.81450625 0.         0.9025     0.        ]
 [0.857375   0.9025     0.95       0.        ]
 [0.         0.95       1.         0.        ]]

Policy:
 [[1. 2. 1. 0.]
 [1. 0. 1. 0.]
 [2. 1. 1. 0.]
 [0. 2. 2. 0.]]


### (b)

In [9]:
# Define the `render` function
def render(policy, env, fname):
    state, prob = env.reset()
    images = []

    while True:
        images.append(env.render())
        action = int(policy[state])
        state, reward, done, info, prob = env.step(action)

        if done:
            break

    imageio.mimsave(fname, images, fps=10)

# Create the FrozenLake environment
env = gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=False, render_mode='rgb_array')

policy_value_iter = policy_improvement(env, V_value_iter)

# Render and save animations for both policies
render(policy_policy_iter, env, 'snap_policy_iter.gif')
render(policy_value_iter, env, 'snap_value_iter.gif')

<h5> Policy Iteration(6 epochs): </h5> <img src="/snap_policy_iter.gif"> 
<h5> Value Iteration(6 epochs): </h5> <img src="/snap_value_iter.gif">

### (c)

So, there is another policy (right, right, down, down, down, right) with same reward as current optimal policy. Both of algorithms did not found this policy.

### (d) (i)

In [10]:
class lake:
    
    def __init__(self, eta=0.8) -> None:
        # Initialize the Lake environment
        self.num_states = 25
        self.num_actions = 4
        self.eta = eta

        # Define the state and action spaces
        self.observation_space = np.arange(self.num_states)
        self.action_space = np.array([0, 1, 2, 3])  # 0->left, 1->down, 2->right, 3->up

        # Define special states and rewards
        self.start = 15
        self.walls = np.array([6, 11, 13])
        self.g1 = 12  # state with +1 exit
        self.g2 = 14  # state with +10 exit

        self.is_reward_set = False
        self.R = np.zeros(25)  # Reward values for states

        self.is_P_set = False
        self.P = {}  # Transition probabilities

        # Initialize rewards and transition probabilities
        self.set_rewards()
        self.set_prob()

    def set_rewards(self):
        # Set the rewards for different states
        if not self.is_reward_set:
            self.R[self.g1] = 1
            self.R[self.g2] = 10

            self.R[20:25] = -10  # Negative rewards

            self.is_reward_set = True

    def set_prob(self):
        # Set transition probabilities
        if not self.is_P_set:
            self.set_rewards()

            actions = [(0, -1), (1, 0), (0, 1), (-1, 0)]

            for i in range(5):
                for j in range(5):
                    curr_state = 5 * i + j
                    self.P[curr_state] = {}

                    for intended_a in self.action_space:
                        temp = []
                        for oth_a in self.action_space:
                            next_i = i + actions[oth_a][0]
                            next_j = j + actions[oth_a][1]
                            prob = 0.0
                            next_state = curr_state

                            if intended_a == oth_a:
                                prob = self.eta  # Transition probability for intended action
                            else:
                                prob = (1 - self.eta) / 3  # Transition probability for other actions

                            if next_i < 0 or next_j < 0 or next_i > 4 or next_j > 4 or 5 * next_i + next_j in self.walls:
                                temp.append((prob, next_state, self.R[next_state]))  # Transition tuple
                            else:
                                next_state = 5 * next_i + next_j  # Update to the new state
                                temp.append((prob, next_state, self.R[next_state]))  # Transition tuple

                        self.P[curr_state][intended_a] = temp  # Store transitions for intended action

        self.is_P_set = True

    def print_grid(self):
        # Print the grid representation of the environment
        print('''\nB-Blank, S-Start, g-Exit with +1 reward, G-Exit with +10 reward, W-Wall, R-Negative states(-10)''')
        for s in range(20):
            i = s // 5
            j = s - i * 5
            if s in self.walls:
                print('W', end=' ')
            elif s == self.start:
                print('S', end=' ')
            elif s == self.g1:
                print('g', end=' ')
            elif s == self.g2:
                print('G', end=' ')
            else:
                print('B', end=' ')
            
            if j % 5 == 4:
                print('')
        
        for s in range(5):
            print('R', end=' ')

        print('\n')


In [11]:
env = lake()
env.set_prob()

env.print_grid()
print(f'Rewards: \n{env.R.reshape(5,5)}\n')
print(f'Transition Probabilities of state 0 with action 1(Down) is: {env.P[0][1]} ')


B-Blank, S-Start, g-Exit with +1 reward, G-Exit with +10 reward, W-Wall, R-Negative states(-10)
B B B B B 
B W B B B 
B W g W G 
S B B B B 
R R R R R 

Rewards: 
[[  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [  0.   0.   1.   0.  10.]
 [  0.   0.   0.   0.   0.]
 [-10. -10. -10. -10. -10.]]

Transition Probabilities of state 0 with action 1(Down) is: [(0.06666666666666665, 0, 0.0), (0.8, 5, 0.0), (0.06666666666666665, 1, 0.0), (0.06666666666666665, 0, 0.0)] 


### (d) (ii)

In [12]:
def policy_improvement(env, V, gamma=0.95):
    # Get the number of states and actions from the environment
    num_states = env.num_states
    num_actions = env.num_actions

    policy = np.zeros(num_states)

    for s in range(num_states):
        q_values = np.zeros(num_actions)

        for a in range(num_actions):
            for (prob, next_state, reward) in env.P[s][a]:
                # Calculate the Q-value for each action using the Bellman equation
                q_values[a] += prob * (reward + gamma * V[next_state])

        best_action = np.argmax(q_values)
        policy[s] = best_action

    return policy

def policy_evaluation(env, policy, gamma=0.95, theta=1e-6):
    # Get the number of states from the environment
    num_states = env.num_states

    V = np.zeros(num_states)

    for epoch in range(max_epochs):
        delta = 0

        for s in range(num_states):
            v = V[s]
            new_v = 0
            a = policy[s]

            for (prob, next_state, reward) in env.P[s][a]:
                # Calculate the new value using the Bellman equation and the policy
                new_v += prob * (reward + gamma * V[next_state])

            V[s] = new_v
            delta = max(delta, abs(v - new_v))

        if delta < theta:
            # Terminate if the value function has converged
            break

    return V

def policy_iteration(env, gamma=0.95):
    # Get the number of states and actions from the environment
    num_states = env.num_states
    num_actions = env.num_actions

    V = np.zeros(num_states)
    policy = np.zeros(num_states)

    for epoch in range(max_epochs):
        V = policy_evaluation(env, policy, gamma)
        new_policy = policy_improvement(env, V, gamma)

        if (np.array_equal(policy, new_policy)):
            print(f'Breaking at epoch: {epoch}')
            break

        policy = new_policy

    return V, policy


In [13]:
V, policy = policy_iteration(env)

env.print_grid()

print('\nValue Function:\n', V.reshape(5,5))
print('\nPolicy:\n', policy.reshape(5,5))

Breaking at epoch: 4

B-Blank, S-Start, g-Exit with +1 reward, G-Exit with +10 reward, W-Wall, R-Negative states(-10)
B B B B B 
B W B B B 
B W g W G 
S B B B B 
R R R R R 


Value Function:
 [[117.57684128 125.91770482 134.89683741 143.91973407 153.41271524]
 [110.31031508 131.26989969 142.0987266  152.64771251 164.29673988]
 [110.19909429 125.51836016 133.84337159 162.24563332 166.45717548]
 [117.43976733 127.39936463 138.33967059 150.42755878 163.18907844]
 [108.40720658 117.18006418 127.41084238 138.66213816 149.77744457]]

Policy:
 [[2. 2. 2. 2. 1.]
 [3. 2. 2. 2. 1.]
 [1. 2. 3. 2. 0.]
 [2. 2. 2. 2. 3.]
 [3. 3. 3. 3. 3.]]


#### Testing for different values of $\eta$ (Above is for $\eta=0.8$ (default))

Test for $\eta=1$

In [14]:
env = lake(eta=1)
V, policy = policy_iteration(env)
env.print_grid()
print('\nValue Function:\n', V.reshape(5,5))
print('\nPolicy:\n', policy.reshape(5,5))

Breaking at epoch: 5

B-Blank, S-Start, g-Exit with +1 reward, G-Exit with +10 reward, W-Wall, R-Negative states(-10)
B B B B B 
B W B B B 
B W g W G 
S B B B B 
R R R R R 


Value Function:
 [[154.75616923 162.90123173 171.47498173 180.49998173 189.99998173]
 [147.01836077 171.47498173 180.49998173 189.99998173 199.99998173]
 [154.75617015 163.90123265 171.47498265 199.99998173 199.99998173]
 [162.90123265 171.47498265 180.49998265 189.99998265 199.99998265]
 [154.75617101 162.90123351 171.47498351 180.49998351 189.99998351]]

Policy:
 [[2. 2. 1. 1. 1.]
 [1. 2. 2. 2. 1.]
 [1. 2. 1. 2. 0.]
 [2. 2. 2. 2. 3.]
 [3. 3. 3. 3. 3.]]


Test for $\eta = 0.5$

In [15]:
env = lake(eta=0.5)
V, policy = policy_iteration(env)
env.print_grid()
print('\nValue Function:\n', V.reshape(5,5))
print('\nPolicy:\n', policy.reshape(5,5))

Breaking at epoch: 2

B-Blank, S-Start, g-Exit with +1 reward, G-Exit with +10 reward, W-Wall, R-Negative states(-10)
B B B B B 
B W B B B 
B W g W G 
S B B B B 
R R R R R 


Value Function:
 [[39.8169965  45.82636322 52.65331472 59.53687285 66.4762782 ]
 [34.36269039 46.38256047 55.45695889 64.40333711 75.78691773]
 [28.85115074 38.04727967 47.88156502 68.9607538  80.13624677]
 [21.42742409 26.59759248 38.17061698 50.21638493 67.68650006]
 [11.27894901 15.9742469  25.42295601 36.23593299 48.12942929]]

Policy:
 [[2. 2. 2. 2. 1.]
 [3. 2. 2. 2. 1.]
 [3. 2. 3. 2. 0.]
 [3. 2. 2. 2. 3.]
 [3. 3. 3. 3. 3.]]


Test for $\eta=0.33$ 

In [16]:
env = lake(eta=0.33)
V, policy = policy_iteration(env)
env.print_grid()
print('\nValue Function:\n', V.reshape(5,5))
print('\nPolicy:\n', policy.reshape(5,5))

Breaking at epoch: 2

B-Blank, S-Start, g-Exit with +1 reward, G-Exit with +10 reward, W-Wall, R-Negative states(-10)
B B B B B 
B W B B B 
B W g W G 
S B B B B 
R R R R R 


Value Function:
 [[  0.61635014   3.93273435   6.80438429  10.55306395  13.96368385]
 [ -4.13873155   2.22881733   5.74048451  11.74914317  18.49894214]
 [-12.14024891 -12.38388043  -3.90169104   7.07009343  18.56037658]
 [-26.82440614 -29.63471212 -21.17383747 -18.55562968  -3.08430858]
 [-41.50681389 -41.40442277 -36.87167374 -32.64460726 -25.34978801]]

Policy:
 [[2. 2. 2. 2. 1.]
 [3. 2. 2. 2. 1.]
 [3. 2. 3. 2. 0.]
 [3. 2. 3. 2. 3.]
 [3. 3. 3. 3. 3.]]


Test for $\eta=0.25$ 

In [17]:
env = lake(eta=0.25)
V, policy = policy_iteration(env)
env.print_grid()
print('\nValue Function:\n', V.reshape(5,5))
print('\nPolicy:\n', policy.reshape(5,5))

Breaking at epoch: 0

B-Blank, S-Start, g-Exit with +1 reward, G-Exit with +10 reward, W-Wall, R-Negative states(-10)
B B B B B 
B W B B B 
B W g W G 
S B B B B 
R R R R R 


Value Function:
 [[-16.03692606 -12.07698549 -10.65957089  -7.45510565  -5.15209467]
 [-23.37306545 -15.61762419 -14.69074418  -8.12314968  -3.93373751]
 [-35.62985299 -38.46698709 -29.43493414 -21.05018279  -9.88044068]
 [-55.38766481 -59.48712114 -52.48121776 -52.77277288 -38.95987033]
 [-72.18026787 -72.58977174 -69.80503746 -67.46097424 -62.42847671]]

Policy:
 [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


* For $\eta=0.8, 1$ algorithm suggesting shortest path from starting state to exit with +10 reward
* For $\eta=0.5, 0.33$ algorithm suggesting longer path from starting state to exit with +10 reward
* For $\eta=0.25$ algorithm suggesting to stay in current state

#### Testing for different values of discount factor, $\gamma$ (Above is for $\gamma=0.95$ (default))

Test for $\gamma=1$

In [18]:
env = lake(eta=0.8)
V, policy = policy_iteration(env, gamma=1)

env.print_grid()

print('\nValue Function:\n', V.reshape(5,5))
print('\nPolicy:\n', policy.reshape(5,5))

Breaking at epoch: 6

B-Blank, S-Start, g-Exit with +1 reward, G-Exit with +10 reward, W-Wall, R-Negative states(-10)
B B B B B 
B W B B B 
B W g W G 
S B B B B 
R R R R R 


Value Function:
 [[4711.3584145  4723.98106404 4736.17101462 4747.49053945 4758.6586508 ]
 [4703.09028861 4732.65447155 4746.1834246  4758.44948544 4770.72738986]
 [4713.53934123 4730.75985477 4741.65986177 4769.32490101 4773.09876399]
 [4725.54815873 4738.83704156 4752.25429465 4765.68884616 4779.12689547]
 [4721.74713967 4734.77564461 4748.16120866 4761.52356323 4773.99634511]]

Policy:
 [[2. 2. 2. 2. 1.]
 [1. 2. 2. 2. 1.]
 [1. 2. 1. 2. 0.]
 [2. 2. 2. 2. 3.]
 [3. 3. 3. 3. 3.]]


Test for $\gamma=0.33$

In [19]:
env = lake(eta=0.8)
V, policy = policy_iteration(env, gamma=0.33)

env.print_grid()

print('\nValue Function:\n', V.reshape(5,5))
print('\nPolicy:\n', policy.reshape(5,5))

Breaking at epoch: 3

B-Blank, S-Start, g-Exit with +1 reward, G-Exit with +10 reward, W-Wall, R-Negative states(-10)
B B B B B 
B W B B B 
B W g W G 
S B B B B 
R R R R R 


Value Function:
 [[ 2.77100143e-02  9.97356793e-02  3.58855032e-01  9.65884153e-01
   3.27884173e+00]
 [ 7.29984753e-03  3.37291797e-01  1.24059327e+00  3.23299157e+00
   1.17928916e+01]
 [-1.53085128e-02  1.14535762e+00  1.26621067e+00  1.16058905e+01
   1.28416203e+01]
 [-7.52822600e-01 -6.26502720e-01  4.59759747e-01  2.27516835e+00
   1.10366710e+01]
 [-2.35315128e+00 -2.31215735e+00 -2.00562923e+00 -1.45520450e+00
   9.22245432e-01]]

Policy:
 [[2. 2. 1. 2. 1.]
 [3. 2. 1. 2. 1.]
 [3. 2. 0. 2. 0.]
 [3. 2. 3. 2. 3.]
 [3. 3. 3. 3. 3.]]


* For $\gamma=0.95,1$, its algorithm is selecting shortest path to exit state with +10 reward
* For $\gamma=0.33$, its algorithm is selecting longer path to exit state with +1 reward

### (d) (iii)

Scaling discount factor $\gamma$ from 0.33 to 0.95 or 1 is changing it's policy. So, scaling $\gamma$ by constant factor may change optimal policy.